project requires python version above than 3.7

In [3]:
import sys
assert sys.version_info >= (3, 7)

In [4]:
IS_COLAB = 'google.colab' in sys.modules

# Get the Data
*In this project you task is to predict the heart patient whether their is chances that he will be cancer patient or not*

## Download the Data

In [5]:
from pathlib import Path
import pandas as pd
import urllib.request


def load_heart_data():
  csv_file_path = Path('datasets/heart.csv')
  if not csv_file_path.is_file():
    Path('datasets').mkdir(parents=True, exist_ok=True)
    # Changed URL to raw content and removed tarfile operations as it's a CSV file
    url = "https://raw.githubusercontent.com/khalidkhankakar/Hands-on-Machine-Learning/master/datasets/heart.csv"
    urllib.request.urlretrieve(url, csv_file_path)
  return pd.read_csv(csv_file_path)

heart_data = load_heart_data()

In [6]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


## Split the dataset

In [10]:
X = heart_data.drop('target', axis=1)
y = heart_data['target'].copy()

In [29]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test, = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)


# Because we can't covert dataframe into tensor directly
X_train, X_valid, X_test, y_train, y_valid, y_test = X_train.values, X_valid.values, X_test.values, y_train.values, y_valid.values, y_test.values

## Convert the data into pytorch tensor

In [31]:
import torch
X_train = torch.FloatTensor(X_train)
X_valid = torch.FloatTensor(X_valid)
X_test = torch.FloatTensor(X_test)

## Scale the data

In [46]:
# Method One: By us StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)

In [48]:
# Method Two: Calculate the Manual method

means = X_train.mean(axis=0, keepdims=True)
stds = X_train.std(axis=0, keepdims=True)

X_train = (X_train - means) / stds
X_valid = (X_valid - means) / stds
X_test = (X_test - means) / stds

In [59]:
y_train = torch.FloatTensor(y_train.reshape(-1, 1))
y_valid = torch.FloatTensor(y_valid.reshape(-1, 1))
x_test = torch.FloatTensor(X_test)

# Make the Model